## Решение инструментом Pandas

In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet('df/couriers_orders.parquet')

#### У компании по доставке еды есть БД в которой содержится таблица заказов пеших курьеров couriers_orders.parquet. 

#### Вопрос №1.1:

В конце каждого месяца компания выдает премию для своих курьеров, средняя скорость доставки за прошедший месяц которых больше средней скорости среди всех курьеров. С
колько курьеров получили премию за июнь 2021 года.

In [3]:
df.head()

,date,courier_id,order_id,distance,travel_time
0,2021-07-12,10,1,1.90,36.17
1,2021-07-02,3,2,3.98,21.34
2,2021-04-15,6,3,3.98,43.33
3,2021-07-16,10,4,2.85,14.01
4,2021-06-11,10,5,4.89,32.09


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1666 entries, 0 to 1665
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         1666 non-null   datetime64[ns]
 1   courier_id   1666 non-null   int64         
 2   order_id     1666 non-null   int64         
 3   distance     1666 non-null   float64       
 4   travel_time  1666 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 65.2 KB


In [5]:
# посмотрим есть ли пропущенные значения
df.isnull().sum()

date           0
courier_id     0
order_id       0
distance       0
travel_time    0
dtype: int64

In [6]:
# создадим копию
df_couriers_t1 = df.copy()

In [7]:
# Сохраним выборку за июнь 2021 года
df_2021_06 = df_couriers_t1[(df_couriers_t1['date'] >= '2021-06-01') & (df_couriers_t1['date'] <= '2021-06-30')]

In [8]:
# Посчитаем и сохраним среднюю скорость доставки в выборкe за июнь 2021 года
mean_tt = df_2021_06['travel_time'].mean()
mean_tt

33.53234482758622

In [9]:
# Выберем курьеров со скоростью доставки выше среднего
couriers_bonus = df_2021_06[df_2021_06['travel_time'] > mean_tt]
couriers_bonus.head()

,date,courier_id,order_id,distance,travel_time
31,2021-06-03,7,32,1.61,57.30
43,2021-06-29,2,44,2.15,57.45
46,2021-06-23,9,47,1.13,37.74
68,2021-06-21,6,69,4.81,37.35
77,2021-06-28,1,78,2.96,53.72


In [10]:
# Выведем ID курьеров со скоростью доставки выше среднего и количество
couriers_bonus['courier_id'].value_counts()

7     19
9     18
5     17
6     15
2     13
4     12
3     12
1     11
10     9
8      9
Name: courier_id, dtype: int64

In [11]:
# Посчитаем уникальные значения
len(couriers_bonus['courier_id'].unique())

10

#### Вопрос №1.2 (используйте данные из предыдущего вопроса №1.1):

Компания хочет понять, насколько равномерно курьеры работают в течение месяца. Для этого нужно найти ID курьера с наибольшей разницей между максимальной и минимальной средней дневной скоростью в июне 2021 года.

In [12]:
# создадим копию
df_couriers_t2 = df.copy()

In [13]:
# Сохраним выборку за июнь 2021 года
df_2021_06_t2 = df_couriers_t2[(df_couriers_t2['date'] >= '2021-06-01') & (df_couriers_t2['date'] <= '2021-06-30')]

In [14]:
# рассчитаем общий пройденный путь и время в движении за каждый рабочий день
result_sum_dis_trav = df_2021_06_t2.groupby(['date','courier_id']).agg({'distance': 'sum', 'travel_time': 'sum'})
result_sum_dis_trav

distance  travel_time
date       courier_id                       
2021-06-01 1               4.21        17.84
           2               4.39        63.24
           3               1.02        48.84
           4               6.78        80.66
           5               5.60        28.92
...                         ...          ...
2021-06-29 10              4.42        99.25
2021-06-30 1               1.59        40.49
           3               4.20        56.51
           5               3.54        23.31
           8               1.69        59.51

[186 rows x 2 columns]

In [15]:
# рассчитаем среднюю дневную скорость для каждого дня и курьера (также преобразуем минуты в часы)
result_sum_dis_trav['mean_speed'] = result_sum_dis_trav.apply(lambda x: x.distance / (x.travel_time / 60), axis=1)
result_sum_dis_trav

distance  travel_time  mean_speed
date       courier_id                                   
2021-06-01 1               4.21        17.84   14.159193
           2               4.39        63.24    4.165085
           3               1.02        48.84    1.253071
           4               6.78        80.66    5.043392
           5               5.60        28.92   11.618257
...                         ...          ...         ...
2021-06-29 10              4.42        99.25    2.672040
2021-06-30 1               1.59        40.49    2.356137
           3               4.20        56.51    4.459388
           5               3.54        23.31    9.111969
           8               1.69        59.51    1.703915

[186 rows x 3 columns]

In [16]:
# сбросим индексы
result_sum_dis_trav.reset_index(inplace=True)

In [17]:
result_sum_dis_trav.head()

,date,courier_id,distance,travel_time,mean_speed
0,2021-06-01,1,4.21,17.84,14.159193
1,2021-06-01,2,4.39,63.24,4.165085
2,2021-06-01,3,1.02,48.84,1.253071
3,2021-06-01,4,6.78,80.66,5.043392
4,2021-06-01,5,5.60,28.92,11.618257


In [18]:
# сгрупируем по курьерам. Расчитаем максимальную и минимальную среднюю дневную скорость для каждого курьера
result = result_sum_dis_trav.groupby(['courier_id']).agg({'mean_speed': ['max', 'min']})
result

mean_speed          
                  max       min
courier_id                     
1           25.342334  1.665531
2           19.404666  1.516789
3           13.351279  1.253071
4           25.776567  0.973960
5           17.651376  1.111456
6           25.497186  1.403068
7           10.115075  1.053556
8           14.946159  0.792570
9           13.315508  0.754786
10          18.605004  1.751201

In [19]:
# вычислите разницу для каждого курьера
result['delta'] = result.mean_speed.apply(lambda x: x['max'] - x['min'], axis=1)
result

mean_speed                delta
                  max       min           
courier_id                                
1           25.342334  1.665531  23.676802
2           19.404666  1.516789  17.887877
3           13.351279  1.253071  12.098207
4           25.776567  0.973960  24.802607
5           17.651376  1.111456  16.539920
6           25.497186  1.403068  24.094118
7           10.115075  1.053556   9.061519
8           14.946159  0.792570  14.153590
9           13.315508  0.754786  12.560722
10          18.605004  1.751201  16.853803

In [20]:
# отсортируем и выведем результат
result['delta'].sort_values(ascending=False)[:1]

courier_id
4    24.802607
Name: delta, dtype: float64

#### Вопрос №2.1:
У нас есть данные о покупках клиентов purchases.parquet. Проанализируйте интервалы времени между последовательными покупками для каждого клиента в наборе данных о покупках - напишите код для вычисления разницы в днях между текущей покупкой и предыдущей покупкой каждого клиента. Отобразите результат в новом столбце days_between_purchases.

In [21]:
df_p = pd.read_parquet('df/purchases.parquet')
df_p.head()

,customer_id,purchase_date
0,2,2021-01-01
1,7,2021-01-01
2,7,2021-01-01
3,11,2021-01-01
4,21,2021-01-01


In [22]:
df_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   customer_id    4000 non-null   int64         
 1   purchase_date  4000 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1)
memory usage: 62.6 KB


In [23]:
# создадим копию
df_interval = df_p.copy()

In [24]:
# отсортируем данные по клиенту и датам покупок
days_between_purchases = df_interval.sort_values(by=['customer_id', 'purchase_date'])
days_between_purchases

,customer_id,purchase_date
149,1,2021-01-14
198,1,2021-01-18
308,1,2021-01-28
386,1,2021-02-05
393,1,2021-02-06
...,...,...
3684,50,2021-12-02
3787,50,2021-12-09
3817,50,2021-12-12
3875,50,2021-12-19


In [25]:
# сгруппируем по клиенту и вычисляем разницу между последовательными покупками
days_between_purchases['interval'] = days_between_purchases.groupby('customer_id')['purchase_date'].diff()
days_between_purchases.head(10)

,customer_id,purchase_date,interval
149,1,2021-01-14,NaT
198,1,2021-01-18,4 days
308,1,2021-01-28,10 days
386,1,2021-02-05,8 days
393,1,2021-02-06,1 days
399,1,2021-02-07,1 days
440,1,2021-02-11,4 days
493,1,2021-02-15,4 days
494,1,2021-02-15,0 days
516,1,2021-02-17,2 days


In [26]:
# посчитаем пустые значения NaT
days_between_purchases.isnull().sum()

customer_id       0
purchase_date     0
interval         50
dtype: int64

In [27]:
# выведем сводную табдицу
pd.pivot_table(data=days_between_purchases,
                     values='interval', 
                     index='customer_id',
                     columns='purchase_date').sample(5)

purchase_date,2021-01-01,2021-01-03,2021-01-04,2021-01-05,2021-01-06,2021-01-07,2021-01-08,2021-01-09,2021-01-10,2021-01-11,...,2021-12-22,2021-12-23,2021-12-24,2021-12-25,2021-12-26,2021-12-27,2021-12-28,2021-12-29,2021-12-30,2021-12-31
customer_id,,,,,,,,,,,,,,,,,,,,,
39,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,5 days,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
46,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2 days,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
14,NaT,NaT,NaT,NaT,NaT,NaT,NaT,1 days,NaT,2 days,...,NaT,NaT,NaT,NaT,NaT,8 days,NaT,NaT,NaT,NaT
27,NaT,NaT,NaT,NaT,NaT,6 days,1 days,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,7 days,NaT,NaT,NaT,NaT
15,NaT,NaT,NaT,3 days,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,3 days,NaT,NaT,NaT,NaT,5 days,NaT,NaT,NaT


#### Вопрос №2.2 (используйте данные из предыдущего вопроса №2.1):

У какого количества уникальных клиентов разница между текущей покупкой и предыдущей покупкой равна 20-ти дням?

In [28]:
days_between_purchases.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4000 entries, 149 to 3900
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype          
---  ------         --------------  -----          
 0   customer_id    4000 non-null   int64          
 1   purchase_date  4000 non-null   datetime64[ns] 
 2   interval       3950 non-null   timedelta64[ns]
dtypes: datetime64[ns](1), int64(1), timedelta64[ns](1)
memory usage: 125.0 KB


In [29]:
# чтобы отсавить только дни, преобразуем timedelta64 в строку и сделаем сплит по 0 индексу
days_between_purchases['days'] = days_between_purchases['interval'].apply(lambda x: str(x).split(' ')[0])

In [30]:
# при смене типа, NaT преобразовался в строку, выберем эти значения и заменим их 0 
days_between_purchases['days'] = days_between_purchases['days'].apply(lambda x: 0 if x == 'NaT' else int(x))

In [31]:
# выберем покупателей разница между текущей покупкой и предыдущей покупкой равна 20-ти дням
res = days_between_purchases[days_between_purchases['days'] == 20]

In [32]:
# выберем уникальные значением и посчтиаем их количество
len(res['customer_id'].unique())

10